## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("../Resources/charity_data.csv")
# application_df = pd.read_csv("/content/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# drop the non-beneficial ID columns, ein and name
# also drop status and special considerations
application_df = application_df.drop(columns=["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"])

In [3]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [4]:
application_df["ASK_AMT"].describe()

count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

In [5]:
application_df["ASK_AMT"].value_counts().loc[lambda x: x > 10]

5000    25398
Name: ASK_AMT, dtype: int64

In [6]:
# bin organization to combine values that have a count of less than 500

org_values_df = application_df["ORGANIZATION"].value_counts().rename_axis('org').reset_index(name='counts')
org_values_df = org_values_df[org_values_df['counts'] < 500]
org_types_to_replace = org_values_df['org'].tolist()

# Replace in dataframe
for org in org_types_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [7]:
# bin use case to combine values that have a count of less than 500

use_values_df = application_df["USE_CASE"].value_counts().rename_axis('use').reset_index(name='counts')
use_values_df = use_values_df[use_values_df['counts'] < 500]
use_types_to_replace = use_values_df['use'].tolist()

# Replace in dataframe
for use in use_types_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(use,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [8]:
# bin affiliation to combine values that have a count of less than 500

aff_values_df = application_df["AFFILIATION"].value_counts().rename_axis('affiliation').reset_index(name='counts')
aff_values_df = aff_values_df[aff_values_df['counts'] < 500]
affiliation_types_to_replace = aff_values_df['affiliation'].tolist()

# Replace in dataframe
for aff in affiliation_types_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [9]:
# bin income amount so that those with 5M+ are grouped together

income_to_replace = ["1M-5M", "5M-10M", "10M-50M", "50M+"]

for income in income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(income, "1M+")
    
income_to_replace2 = ["1-9999", "10000-24999"]
for income in income_to_replace2:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(income, "1-24999")

application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M+               1519
1-24999           1271
Name: INCOME_AMT, dtype: int64

In [10]:
# bin application type to combine values that have a count of less than 500

values_df = application_df["APPLICATION_TYPE"].value_counts().rename_axis('app_types').reset_index(name='counts')
values_df = values_df[values_df['counts'] < 500]
application_types_to_replace = values_df['app_types'].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# bin classification to combine values that have a count of less than 1000

class_values_df = application_df["CLASSIFICATION"].value_counts().rename_axis('classification').reset_index(name='counts')
class_values_df = class_values_df[class_values_df['counts'] < 1000]
classifications_to_replace = class_values_df['classification'].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Other,1-24999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1-24999,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,142590,1


In [13]:
# adjust ask amount
# using application_df["ASK_AMT"].describe() before our preprocessing,
# we see that the 75% value of the distribution is about $7742
# the interquartile range is $2742, so outliers would be considered to be those data points that are greater than $10,484
# about one quarter of our data is thus considered to be an outlier
# thus, we will test with different amounts to see what would be a good boundary, starting with 10,000

application_df = application_df[application_df["ASK_AMT"] < 10000]
# Loss: 0.5328684449195862, Accuracy: 0.7494990229606628

# application_df = application_df[application_df["ASK_AMT"] < 100000]
# results: Loss: 0.5567649006843567, Accuracy: 0.7422735691070557

# application_df = application_df[application_df["ASK_AMT"] < 1000000]
# Loss: 0.5561736822128296, Accuracy: 0.7346290349960327

# application_df = application_df[application_df["ASK_AMT"] < 10000000]
# Loss: 0.5559248924255371, Accuracy: 0.7258026599884033

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_data = pd.get_dummies(application_df)
# dummy_data.columns

In [15]:
# Split our preprocessed data into our features and target arrays
y = dummy_data["IS_SUCCESSFUL"].values
X = dummy_data.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1872)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
layer1_nodes = 61
layer2_nodes = 16
layer3_nodes = 26
layer4_nodes = 26

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1_nodes, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2_nodes, activation="relu"))
nn.add(tf.keras.layers.Dense(units=layer3_nodes, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=layer4_nodes, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 61)                1891      
                                                                 
 dense_1 (Dense)             (None, 16)                992       
                                                                 
 dense_2 (Dense)             (None, 26)                442       
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 3,352
Trainable params: 3,352
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
609/609 [==============================] - 1s 599us/step - loss: 0.5573 - accuracy: 0.7311
Epoch 2/100
609/609 [==============================] - 0s 581us/step - loss: 0.5363 - accuracy: 0.7448
Epoch 3/100
609/609 [==============================] - 0s 583us/step - loss: 0.5342 - accuracy: 0.7463
Epoch 4/100
609/609 [==============================] - 0s 574us/step - loss: 0.5324 - accuracy: 0.7474
Epoch 5/100
609/609 [==============================] - 0s 574us/step - loss: 0.5323 - accuracy: 0.7474
Epoch 6/100
609/609 [==============================] - 0s 571us/step - loss: 0.5313 - accuracy: 0.7479
Epoch 7/100
609/609 [==============================] - 0s 572us/step - loss: 0.5309 - accuracy: 0.7489
Epoch 8/100
609/609 [==============================] - 0s 572us/step - loss: 0.5299 - accuracy: 0.7495
Epoch 9/100
609/609 [==============================] - 0s 570us/step - loss: 0.5298 - accuracy: 0.7490
Epoch 10/100
609/609 [==============================] - 0s 570us/step - l

609/609 [==============================] - 0s 682us/step - loss: 0.5217 - accuracy: 0.7534
Epoch 80/100
609/609 [==============================] - 0s 721us/step - loss: 0.5223 - accuracy: 0.7533
Epoch 81/100
609/609 [==============================] - 0s 729us/step - loss: 0.5220 - accuracy: 0.7536
Epoch 82/100
609/609 [==============================] - 0s 705us/step - loss: 0.5215 - accuracy: 0.7535
Epoch 83/100
609/609 [==============================] - 0s 770us/step - loss: 0.5216 - accuracy: 0.7533
Epoch 84/100
609/609 [==============================] - 0s 739us/step - loss: 0.5216 - accuracy: 0.7536
Epoch 85/100
609/609 [==============================] - 0s 727us/step - loss: 0.5210 - accuracy: 0.7548
Epoch 86/100
609/609 [==============================] - 0s 718us/step - loss: 0.5210 - accuracy: 0.7529
Epoch 87/100
609/609 [==============================] - 0s 722us/step - loss: 0.5209 - accuracy: 0.7543
Epoch 88/100
609/609 [==============================] - 0s 765us/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

203/203 - 0s - loss: 0.5343 - accuracy: 0.7486 - 331ms/epoch - 2ms/step
Loss: 0.534260094165802, Accuracy: 0.7485740780830383


In [21]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu', 'tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=81,
        step=10), activation=activation, input_dim=len(X_train.columns)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [22]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    overwrite=True,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [23]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 72 Complete [00h 00m 06s]
val_accuracy: 0.7508863806724548

Best val_accuracy So Far: 0.7536611557006836
Total elapsed time: 00h 04m 31s

Search: Running Trial #73

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
51                |81                |first_units
3                 |2                 |num_layers
26                |11                |units_0
16                |16                |units_1
26                |6                 |units_2
1                 |21                |units_3
50                |50                |tuner/epochs
17                |17                |tuner/initial_epoch
2                 |3                 |tuner/bracket
2                 |3                 |tuner/round
0069              |0049              |tuner/trial_id

Epoch 18/50
609/609 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7348 - val_loss: 0.5303 - val_accuracy: 0.7487
Epoch 19/50
609/609 [===============

KeyboardInterrupt: 

use above results to make model with target accuracy

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
layer1_nodes = 81
layer2_nodes = 11

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1_nodes, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2_nodes, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 81)                2511      
                                                                 
 dense_6 (Dense)             (None, 11)                902       
                                                                 
 dense_7 (Dense)             (None, 1)                 12        
                                                                 
Total params: 3,425
Trainable params: 3,425
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
609/609 [==============================] - 1s 731us/step - loss: 0.5607 - accuracy: 0.7280
Epoch 2/100
609/609 [==============================] - 0s 676us/step - loss: 0.5403 - accuracy: 0.7436
Epoch 3/100
609/609 [==============================] - 0s 723us/step - loss: 0.5357 - accuracy: 0.7445
Epoch 4/100
609/609 [==============================] - 0s 703us/step - loss: 0.5354 - accuracy: 0.7429
Epoch 5/100
609/609 [==============================] - 0s 699us/step - loss: 0.5332 - accuracy: 0.7437
Epoch 6/100
609/609 [==============================] - 0s 720us/step - loss: 0.5322 - accuracy: 0.7461
Epoch 7/100
609/609 [==============================] - 0s 687us/step - loss: 0.5321 - accuracy: 0.7464
Epoch 8/100
609/609 [==============================] - 0s 660us/step - loss: 0.5311 - accuracy: 0.7476
Epoch 9/100
609/609 [==============================] - 0s 688us/step - loss: 0.5296 - accuracy: 0.7501
Epoch 10/100
609/609 [==============================] - 0s 727us/step - l

609/609 [==============================] - 0s 688us/step - loss: 0.5196 - accuracy: 0.7549
Epoch 80/100
609/609 [==============================] - 0s 665us/step - loss: 0.5195 - accuracy: 0.7550
Epoch 81/100
609/609 [==============================] - 0s 740us/step - loss: 0.5194 - accuracy: 0.7548
Epoch 82/100
609/609 [==============================] - 0s 745us/step - loss: 0.5196 - accuracy: 0.7550
Epoch 83/100
609/609 [==============================] - 0s 738us/step - loss: 0.5194 - accuracy: 0.7549
Epoch 84/100
609/609 [==============================] - 0s 775us/step - loss: 0.5195 - accuracy: 0.7549
Epoch 85/100
609/609 [==============================] - 0s 735us/step - loss: 0.5194 - accuracy: 0.7547
Epoch 86/100
609/609 [==============================] - 0s 757us/step - loss: 0.5193 - accuracy: 0.7549
Epoch 87/100
609/609 [==============================] - 0s 754us/step - loss: 0.5193 - accuracy: 0.7551
Epoch 88/100
609/609 [==============================] - 0s 778us/step - loss:

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

203/203 - 0s - loss: 0.5256 - accuracy: 0.7507 - 313ms/epoch - 2ms/step
Loss: 0.5255534052848816, Accuracy: 0.7507322430610657


**Original**

268/268 - 1s - loss: 0.5656 - accuracy: 0.7251 - 509ms/epoch - 2ms/step

Loss: 0.5656004548072815, Accuracy: 0.7251312136650085



**Final optimization**

2203/203 - 0s - loss: 0.5256 - accuracy: 0.7507 - 313ms/epoch - 2ms/step

Loss: 0.5255534052848816, Accuracy: 0.7507322430610657

In [30]:
# Export our model to HDF5 file
nn.save("/content/nn_model.hdf5")